# Fase 5 - Interface Grafica
Vamos agora pegar tudo que aprendemos e juntar em uma interface grafica utilizavel.
Transformando nossos codigos em funções a serem chamadas.


In [58]:
from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime
import webbrowser


### Metodos

#### Coleta de Dados.

In [59]:
def pesquisar(Produto):
    produto = str(Produto).upper()
    listaX = []
    driver = webdriver.Chrome(executable_path= './ignore/chromedriver.exe')
    # Entrando no catalogo da magalu
    url = 'https://www.magazineluiza.com.br/busca/{}/'.format(produto)
    driver.get(url)
    #Raspando os elementos
    linha = 1
    html = '/html/body/div[1]/div/main/section[4]/div[3]/div/ul/'
    while True:
        try:
            html = '/html/body/div[1]/div/main/section[4]/div[3]/div/ul/'
            listaInterna = []
            #Coletando os valores da pagina
            try: # PRODUTO
                nome = driver.find_element(By.XPATH,html +'li[{}]/a/div[3]/h2'.format(linha)).text
            except:
                html = '/html/body/div[1]/div/main/section[4]/div[4]/div/ul/'
                nome = driver.find_element(By.XPATH,html +'li[{}]/a/div[3]/h2'.format(linha)).text
            try: # PREÇO
                price = driver.find_element(By.XPATH, html +'li[{}]/a/div[3]/div[2]/div/div/p'.format(linha)).text
            except:
                price = driver.find_element(By.XPATH, html +'li[{}]/a/div[3]/div[1]/div/div/p'.format(linha)).text
                #Link para compra    
            link = driver.find_element(By.XPATH,html +'li[{}]/a'.format(linha)).get_attribute('href')
            
            listaInterna.append(nome)
            listaInterna.append(price)
            listaInterna.append(link)
            listaX.append(listaInterna) 
            #print('Tudo Ok')
            linha = linha + 1      
            if linha == 10: break
        except:
            if linha == 1: print("Magualu is Fail")
            break
    #Convertendo Price to float
    y = 0
    for x in listaX:
        limpeza = x[1][3:].replace('.','')
        listaX[y][1] = float(limpeza.replace(',','.'))
        #print(listaX[y][1]) 
        y += 1
    #Menor
    produtoHoje = []
    for x in listaX:
        if produtoHoje == []: produtoHoje = x 
        if produtoHoje[1] > x[1]: produtoHoje = x 

    # Obter a data e hora atuais
    data_hora_atual = datetime.now()
    # Obter os componentes da data e hora

    produtoHoje.append(f"{data_hora_atual.day}/{data_hora_atual.month}/{data_hora_atual.year}")
    return produtoHoje
    
        

#### Json stufs.

In [60]:
    # novo_objeto = json.dumps(menor)
    # Aqui abrimos o nosso arquivo para poder mecher nele
def salvajson(self, target = "logFase5.json", produto = "Celular",produtoHoje = []):        
        with open(target, 'r') as arquivo:
            dados = json.load(arquivo)
        # -----------------------------------------------------------------------------------

        if produto in dados: #Varrendo nossos dados para encontrar nosso produto:
            #Produto Lista existe agora vamos adicionar o produto de Hoje.
            validar = True # Aqui o validar vai ser positivo para adicionar o novo elemento a lista.
                #Se encontrado enquanto vasculhamos Validar sera neativado e não vamos repetir produtos.
            for sub_cell in dados[produto]: #Tentando verificar se o produto de hj existe
                if sub_cell[3] == produtoHoje[3]:
                    print('Repitido')
                    validar = False
                    break
            if validar: dados[produto].append(produtoHoje)
        else: # Não existe a lista do produto então a criamos
            dados[produto] = [produtoHoje]
            
        # Salvando
        with open("logFase5.json", "w") as arquivo:
            json.dump(dados, arquivo, indent=2)
        
        if False: #Só vai escrever
            with open('logFase5.json', 'w') as file:
            #dados.append(novo_objeto)
                json.dump(menor, file, indent=4)

        


In [61]:
    # Retornando o menor Valor
def consultarHistorico(produto):
        with open('logFase4.json', 'r') as arquivo:
            dados = json.load(arquivo)
        menor = []
        for x in dados[produto]:
            if menor == []: menor = x 
            if menor[1] > x[1]: menor = x
        print(menor)

### Interface Grafica
Vamos la montar isso, vamos fazer simples, uma tela com algumas opções:

Uma barra para colocar o nome do produto.

Depois de Enviado ele retorna o produto com nome, preço e link

In [62]:
import PySimpleGUI as sg
import webbrowser

def janela_main():
    layout = [
        [sg.Text("Digite o nome do produto:")],
        [sg.Input(key="-NOME-")],
        [sg.Button("Buscar", key="Enviar"),
            [sg.Button("Visualizar Ultimo Produto", key="-BOTAO_HABILITADO-", visible=False, disabled=True)]
        ]
    ]
    window = sg.Window("Caçador de Preços", layout,size=(400, 200))
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == "Enviar": # Aqui vamos adicionar o chamado aos nossos metodos
            produtohoje = pesquisar(values["-NOME-"])
            produto = values["-NOME-"]
            
            #nome_digitado = values["-NOME-"]
            window.hide()  
            janela_produto(produto, produtohoje)
            window["-BOTAO_HABILITADO-"].update(visible=True, disabled=False)

            window.un_hide()
    window.close()


#Janela onde sera exibido os produtos
def janela_produto( produto, produtohoje):
    layout = [
        [sg.Text("Resultado da busca")],
        [sg.Text(f"Nome digitado: {produto} \nModelo encontrado:"+
            f"{produtohoje[0][15:]}\nR${produtohoje[1]}"        
        )],
        [sg.Button("Abrir Link"), sg.Button("Voltar")],
        
    ]

    window = sg.Window("Busca Efetuada", layout,size=(400, 200))

    while True:
        event, _ = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == "Voltar":
            window.close()  # Fecha a janela secundária e retorna para a janela principal
            break
        if event == "Abrir Link":
            link = produtohoje[2]
            webbrowser.open(link)


if True:
    janela_main()


C:\Users\berna\AppData\Local\Temp\ipykernel_8928\2499439951.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= './ignore/chromedriver.exe')


Link Open


In [ ]:
import PySimpleGUI as sg
import webbrowser

# Sua função que retorna o link da internet
def obter_link():
    return "https://www.example.com"  # Substitua isso pelo link retornado pela sua função

def main():
    layout = [
        [sg.Button("Abrir Link")]
    ]

    window = sg.Window("Abrir Link no Navegador", layout)

    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == "Abrir Link":
            link = obter_link()
            if link:
                webbrowser.open(link)

    window.close()

if __name__ == "__main__":
    main()
